In [8]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import talib as ta

def make_time_series(dataname, filename, number):
    df = pd.read_csv(dataname, index_col='Datetime')
    df = add_technical(df, 0)
    for i in range(number):
        df_temp = pd.read_csv(dataname, index_col='Datetime')
        df_temp = df_temp.shift(i+1)
        df_temp = add_technical(df_temp)
        df = pd.merge(df_temp, df, on='Datetime') 
    
    df = df.dropna(how="any", axis=0)
    df.to_csv(filename, mode = 'w')
    pd.set_option('display.max_columns', 100)
    print(df)
    
def add_technical(df, num=1):
    
    df_feature = pd.DataFrame() #特徴量を入れるための空のdataframeを作成
    
    if num==0: #最新のデータのみrateを算出
        df_feature["Rate"] = df["Close"]-df["Open"]
        df_feature["Rate"] = df_feature["Rate"].shift(-1)
    close = np.array(df["Close"]) #後の計算用に取り出しておく
    df_feature['Volume'] = df['Volume']
    #以下、talibを用いてテクニカル指標（今回の学習で用いる特徴量）を算出しdf_feature入れる
    #単純移動平均は、単純移動平均値とその日の終値の比を特徴量として用いる
    df_feature["SMA/current"]= ta.SMA(close, timeperiod=5) / close
    #RSI
    df_feature["RSI"] = ta.RSI(close, timeperiod=12)
    #MACD
    df_feature["MACD"], _ , _= ta.MACD(close, fastperiod=3, slowperiod=18, signalperiod=9)
    #ボリンジャーバンド 
    upper, middle, lower = ta.BBANDS(close, timeperiod=20, nbdevup=3, nbdevdn=3)
    df_feature["BBANDS+2σ"] = upper / close
    df_feature["BBANDS-2σ"] = lower / close
    df_feature["ROC"] = ta.ROCP(close) / close
    return df_feature

filename = '20101to202010_USD_JPY_D'   ##.csvは除くこと

make_time_series("./"+filename+".csv", filename + "_series.csv", 5)

                  Volume_x  SMA/current_x      RSI_x    MACD_x  BBANDS+2σ_x  \
Datetime                                                                      
2010-01-31 22:00   34533.0       1.001201  35.040215 -0.839712     1.042556   
2010-01-31 22:00   34533.0       1.001201  35.040215 -0.839712     1.042556   
2010-01-31 22:00   34533.0       1.001201  35.040215 -0.839712     1.042556   
2010-01-31 22:00   34533.0       1.001201  35.040215 -0.839712     1.042556   
2010-01-31 22:00   34533.0       1.001201  35.040215 -0.839712     1.042556   
...                    ...            ...        ...       ...          ...   
2020-10-22 21:00   60291.0       0.999647  47.091802 -0.115066     1.007830   
2020-10-25 21:00   53223.0       0.999501  47.886030 -0.086827     1.006625   
2020-10-26 21:00   57498.0       0.998961  49.332152 -0.045545     1.006023   
2020-10-27 21:00   70716.0       1.006561  33.973136 -0.384285     1.016932   
2020-10-28 21:00   55517.0       1.002976  39.684816